In [ ]:
import rasterio

from rasterio.plot import show
from palettable.colorbrewer.diverging import RdBu_5 as RedBlueCmap
from mpl_toolkits.axes_grid1 import make_axes_locatable

RED_BLUE_CMAP = RedBlueCmap.mpl_colormap
from common import *

plt.rcParams.update(
    {
        'xtick.labelsize' : 10,
        'ytick.labelsize' : 10,
    }
)

## Snow Depth comparison 

In [ ]:
differences = {}
dates = ['0331', '0524']
FILE_PREFIX = 'depth_difference_50m_2018'
home = PurePath(f'{Path.home()}/scratch/iSnobal/2018-ASO-data/Depth/')

In [ ]:
for date in dates:
    snow_nc = f'{FILE_PREFIX}{date}.tif'
    file = rasterio.open(home.joinpath(snow_nc).as_posix())
    differences[date] = file.read(1, masked=True).compressed()

In [ ]:
def violin_plot(ax, data, title, color):
    vp = ax.violinplot(
        data,
        showmeans=False, showextrema=False, showmedians=True,
    )
    ax.set_xticks([])
    ax.axhline(0, c='black', ls=':')
    
    for pc in vp['bodies']:
        pc.set_facecolor(color)
        pc.set_edgecolor('black')
    
    vp['cmedians'].set_color('black')

    return vp

In [ ]:
def area_plot(date):
    snow_nc = f'{FILE_PREFIX}{date}.tif'
    file = rasterio.open(home.joinpath(snow_nc).as_posix())
    data = file.read(1, masked=True)

    fig, (ax1, ax2) = plt.subplots(
        1, 2, figsize=(7, 3.5),
        gridspec_kw={'width_ratios': [7, 3], 'wspace': 0.2},
        dpi=300,
    )
    fig.set_facecolor('lightgrey')
    ax1.set_facecolor('lightslategrey')

    plt_data = ax1.imshow(
        data,
        vmin=-2,
        vmax=2,
        cmap=RED_BLUE_CMAP
    )
    show(
        data,
        ax=ax1,
        transform=file.transform, 
        vmin=-2,
        vmax=2,
        cmap=RED_BLUE_CMAP
    )
    ax1.tick_params(axis='both', direction='inout', length=7, labelsize=9, pad=0)
    ax1.tick_params(axis='y', labelrotation=45)
    ax1.ticklabel_format(axis='both', style='plain')
    ax1.xaxis.set_ticks_position('both')
    ax1.yaxis.set_ticks_position('both')
    
    cax = make_axes_locatable(ax1).append_axes('right', size='5%', pad=0.15)
    fig.colorbar(plt_data, cax=cax, shrink=0.9, extend='both', label=r'$\Delta$ Depth (m)')
    fig.suptitle(datetime.datetime.strptime(f"2018{date}", '%Y%m%d').date().isoformat(), y=0.94)
        
    violin_plot(ax2, differences[date], date, 'orange')
    ax2.set_ylabel(None)
    ax2.set_ylim([-2, 2])
    ax2.tick_params(axis='y', labelsize=9)
    plt.savefig(f"/data/iSnobal/figures/ASO_depth_difference_{date}.png")


In [ ]:
for date in dates:
    area_plot(date)
    print(f'Median - {date}: {np.median(differences[date]):.2f}')
